## Expert Knowledge Worker
+ A question answering agent that is an expert knowledge worker.
+ To be used by Insurellm, an Insurance Tech Company.
+ The agent needs to be accurate and the solution should be low cost.
---
This project will use RAG to ensure our question/answering assistant's high accuracy.  


In [1]:
import os
import glob
from dotenv import load_dotenv
import gradio as gr

In [2]:
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain


In [3]:
load_dotenv(override=True)

MODEL = 'gpt-4o-mini'
db_name = "vector_db"

In [4]:
# Step 1. Loading the documents using LangChain loaders

text_loader_kwargs = {'encoding': 'utf-8'}
documents = []

folders = glob.glob("knowledge-base/*")
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata['doc_type'] = doc_type
        documents.append(doc)

In [5]:
# Step 2. Chunking

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

Created a chunk of size 1088, which is longer than the specified 1000


In [6]:
# Step 3. Embedding and putting the chunks in Vector Database

embeddings = OpenAIEmbeddings()

# is db already exists, delete it 
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

# Create Vector Store
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vector DB Created with {vectorstore._collection.count()} documents")



Vector DB Created with 123 documents


In [7]:
# collecting one documents and finding how many dimensions it has !!

collection = vectorstore._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions} dimensions.")

The vectors have 1536 dimensions.


### **Time to bring it all together using LangChain !!**

In [8]:
# The final step 

# Create a new chat model with OpenAI
llm = ChatOpenAI(temperature=0.7, model=MODEL)

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# The retriever is an abstraction over the vector store that will be used during RAG
retriever = vectorstore.as_retriever()

# putting it all together: set up the conversation chain with the gpt-4.1-nano llm, vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

C:\Users\KESHAV\AppData\Local\Temp\ipykernel_9152\1927888662.py:7: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [ ]:
# testing out !!

query = "Can you describe Insurellm in few sentences ?"
result = conversation_chain.invoke({'question': query})

print(result['answer'])


### Gradio UI !!

In [48]:
def chat(query, history):
    result = conversation_chain.invoke({'question': query})
    return result['answer']

In [49]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


#### As the LangChain `ConversationBufferMemory()` and all similar packages have been deprecated, here's an alternative !!

In [10]:
from langgraph.checkpoint.memory import MemorySaver 
from langgraph.prebuilt import create_react_agent
from IPython.display import Markdown

In [ ]:
# Setting up the LLM
llm = ChatOpenAI(temperature=0.7, model=MODEL)

# Setting up the memory
memory = MemorySaver()

# Reteriever object
retriever = vectorstore.as_retriever()

# to be passed as a tool to the agent
retriever_tool = retriever.as_tool(
    name="retriever", 
    description="Retrieve relevant documents from the knowledge base."
)

# Agent creation
agent = create_react_agent(
    model=llm, 
    tools=[retriever_tool], 
    checkpointer=memory
)

# configurables
config = {'configurable': {'thread_id': '1'}}

C:\Users\KESHAV\AppData\Local\Temp\ipykernel_23972\3651799587.py:7: LangChainBetaWarning: This API is in beta and may change in the future.
  retriever_tool = retriever.as_tool(


In [16]:
# Chat Function for gradio UI

def chat_with_agent(message, history):
    """
    Simple chat function for the React agent
    """
    
    # Create the user message
    user_message = {'role': 'user', 'content': message}
        
    # Get response from agent
    result = agent.invoke({'messages': [user_message]}, config)
        
    # Extract the assistant's response
    response = result['messages'][-1].content
        
    return response


gr.ChatInterface(fn=chat_with_agent, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
